In [18]:
!pip install evaluate seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=adc0c0fa8e9de4f8516d2b053d6ca4179c072da84ec64047b19b27f77ca67c17
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
# Load the Pretrained Model

model_name = "rasyosef/bert-tiny-amharic"

from transformers import AutoTokenizer

# Load tiny Amharic tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)



tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/274k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/725k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [29]:
def tokenize_and_align(batch):
    tokenized_inputs = tokenizer(batch["tokens"], is_split_into_words=True, truncation=True, padding=True)

    all_labels = []
    for i in range(len(batch["tokens"])):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                labels.append(-100)
            elif word_idx != previous_word_idx:
                labels.append(batch["labels"][i][word_idx])
            else:
                labels.append(batch["labels"][i][word_idx])  # or -100 for subwords
            previous_word_idx = word_idx

        # Pad labels to the same length as input_ids
        padding_length = tokenized_inputs['input_ids'][i].__len__() - len(labels)
        labels += [-100] * padding_length
        all_labels.append(labels)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

# Now apply
tokenized_dataset = dataset.map(tokenize_and_align, batched=True)

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [37]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./amharic-ner-bert-tiny",     # where to save model checkpoints
    eval_strategy="epoch",                   # evaluate after each epoch
    learning_rate=2e-5,                       # small learning rate for transformers
    per_device_train_batch_size=8,           # depends on your RAM/VRAM
    per_device_eval_batch_size=8,
    num_train_epochs=5,                      # increase if underfitting
    weight_decay=0.01,                        # regularization
    save_strategy="epoch",                    # save model each epoch
    logging_dir="./logs",                     # logs for TensorBoard (optional)
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_overall_f1",              # will define this in compute_metrics
)

In [38]:
from transformers import AutoModelForTokenClassification

# Load the pretrained model with the correct number of labels
model = AutoModelForTokenClassification.from_pretrained(
    "rasyosef/bert-tiny-amharic",
    num_labels=len(tag2id),
    id2label=id2tag,
    label2id=tag2id
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at rasyosef/bert-tiny-amharic and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
import evaluate

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_preds = [
        [id2tag[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    return seqeval.compute(predictions=true_preds, references=true_labels)


In [40]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-40-1058956260.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Loc,Product,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,No log,0.973040,"{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}",0.000000,0.000000,0.000000,0.774194
2,No log,0.964326,"{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}",0.000000,0.000000,0.000000,0.774194
3,No log,0.957907,"{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}",0.000000,0.000000,0.000000,0.806452
4,No log,0.953713,"{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}",0.000000,0.000000,0.000000,0.806452
5,1.031700,0.951960,"{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}",0.000000,0.000000,0.000000,0.838710


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0}" of type <class 'dict'> for key "eval/Product" as a scalar. This invocation of T

TrainOutput(global_step=10, training_loss=1.0317463874816895, metrics={'train_runtime': 3.8353, 'train_samples_per_second': 11.733, 'train_steps_per_second': 2.607, 'total_flos': 10938529980.0, 'train_loss': 1.0317463874816895, 'epoch': 5.0})

In [42]:
trainer.save_model("amharic_ner_bert_tiny_model")
tokenizer.save_pretrained("amharic_ner_bert_tiny_model")


('amharic_ner_bert_tiny_model/tokenizer_config.json',
 'amharic_ner_bert_tiny_model/special_tokens_map.json',
 'amharic_ner_bert_tiny_model/vocab.txt',
 'amharic_ner_bert_tiny_model/added_tokens.json',
 'amharic_ner_bert_tiny_model/tokenizer.json')